<a href="https://colab.research.google.com/github/DannyTheFlower/PDA_Course/blob/main/labs/lab_1/PDA_L1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание на лабораторную работу 1

Представим задачу предсказания оценки как задачу регрессии, где целевой переменной будет признак Book-Rating. Тогда план анализа данных будет следующим:
1. Предобработать данные: преобразовать все категориальные данные в числовые (для преобразования названий книжек в вектор попробуйте не менее 3 подходов из перечисленных: bag-of-words, tf-idf, word2vec, fasttext), обработать пустые ячейки и неточности в столбцах.
2. Сделать нормировку данных.
3. Обучить и протестировать не менее трех моделей регрессии. Например, линейную регрессию, случайный лес и xgboost. Можно также попробовать применить нейронную сеть.
4. Оценить результаты.


# Загрузка данных и их подготовка

In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
!mkdir /root/.kaggle
!touch /root/.kaggle/kaggle.json
!echo '{"username":"daniilpiatygo","key":"6a5c15c37176a5dd0af08791f3882e69"}' > /root/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d arashnic/book-recommendation-dataset
!unzip -q book-recommendation-dataset.zip

 21% 5.00M/23.8M [00:00<00:00, 42.8MB/s]
100% 23.8M/23.8M [00:00<00:00, 118MB/s] 


In [4]:
import numpy as np
import pandas as pd
import gc

In [5]:
books = pd.read_csv("Books.csv")
ratings = pd.read_csv("Ratings.csv")
users = pd.read_csv("Users.csv")

books = books.merge(ratings, on="ISBN").merge(users, on="User-ID")

In [6]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating,Location,Age
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2,0,"stockton, california, usa",18.0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,5,"timmins, ontario, canada",NaN
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,8,0,"timmins, ontario, canada",NaN
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,8,0,"timmins, ontario, canada",NaN
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,8,0,"timmins, ontario, canada",NaN


Удалим неиспользуемые переменные

In [7]:
del ratings, users
gc.collect()

0

Преодобработку возьмём из [второй практики](https://github.com/DannyTheFlower/PDA_Course/blob/main/practices/practice_2/PDA_P2.ipynb).

In [8]:
locations = books["Location"].str.split(", ", expand=True)
locations.drop(range(3, 7), axis=1, inplace=True)
locations.columns = ["city", "region", "country"]
books = pd.concat([books, locations], axis=1)

books["Age"] = books["Age"].apply(lambda x: x if 0 <= x <= 100 else np.nan)

condition = (books["Year-Of-Publication"] == "Gallimard") | (books["Year-Of-Publication"] == "DK Publishing Inc")
books.loc[condition, ["Book-Author", "Year-Of-Publication"]] = books.loc[condition, ["Year-Of-Publication", "Book-Author"]].values

In [9]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating,Location,Age,city,region,country
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2,0,"stockton, california, usa",18.0,stockton,california,usa
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,5,"timmins, ontario, canada",NaN,timmins,ontario,canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,8,0,"timmins, ontario, canada",NaN,timmins,ontario,canada
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,8,0,"timmins, ontario, canada",NaN,timmins,ontario,canada
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,8,0,"timmins, ontario, canada",NaN,timmins,ontario,canada


Удалим неиспользуемые переменные

In [10]:
del locations, condition
gc.collect()

18

Посмотрим, что можно ещё сделать для обучения моделей.

In [11]:
books.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1031136 entries, 0 to 1031135
Data columns (total 15 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   ISBN                 1031136 non-null  object 
 1   Book-Title           1031136 non-null  object 
 2   Book-Author          1031135 non-null  object 
 3   Year-Of-Publication  1031136 non-null  object 
 4   Publisher            1031134 non-null  object 
 5   Image-URL-S          1031136 non-null  object 
 6   Image-URL-M          1031136 non-null  object 
 7   Image-URL-L          1031132 non-null  object 
 8   User-ID              1031136 non-null  int64  
 9   Book-Rating          1031136 non-null  int64  
 10  Location             1031136 non-null  object 
 11  Age                  750391 non-null   float64
 12  city                 1031136 non-null  object 
 13  region               1031136 non-null  object 
 14  country              1006941 non-null  object 
dty

Есть проблема с признаком `Year-Of-Publication`: его тип `object`, а не `int64`. Эту проблему мы решили во второй практической работе, поэтому просто преобразуем его к нужному типу.

In [12]:
books["Year-Of-Publication"] = pd.to_numeric(books["Year-Of-Publication"])

Поработаем с категориальными признаками. В рамках данной лабораторной работы мы не будем как-либо использовать ссылки на картинки, поэтому их просто удалим.

In [13]:
books.drop(columns=["Image-URL-S", "Image-URL-M", "Image-URL-L"], inplace=True)

`ISBN` будем кодировать с помощью `LabelEncoder`.

In [14]:
from sklearn.preprocessing import LabelEncoder

books["ISBN"] = LabelEncoder().fit_transform(books["ISBN"])

Поработаем с пропусками. Признаки `Book-Author` и `Publisher` содержат в себе 1 и 2 пропуска соответственно — на весь массив данных это пренебрежительно малая часть, поэтому их можно удалить.

In [15]:
books.dropna(subset=["Book-Author", "Publisher"], inplace=True)

`Age` имеет значительное количество пропусков. Заменим их на некоторое усреднённое: среднее или медиану. Посмотрим на распределение этого признака, чтобы выбрать более удачный вариант.

In [16]:
books["Age"].describe()

count    750388.000000
mean         36.993068
std          12.220704
min           0.000000
25%          28.000000
50%          35.000000
75%          45.000000
max         100.000000
Name: Age, dtype: float64

Разница между средним и медианой в целом незначительна — почти 2 года при разбросе возрастов от 0 до 100. Остановимся на медиане.

In [17]:
books["Age"] = books["Age"].fillna(books["Age"].median())

Также необходимо заполнить пропуски в столбце `country`. Посмотрим на несколько уникальных значений, которые были обработаны неправильно.

In [18]:
books[books["country"].isna()]["Location"].unique()[:10]

array(['ottawa, ,', 'seattle, ,', 'albuquerque, ,', 'humble, ,',
       'springfield, ,', 'aloha, ,', 'pearland, texas,', 'sarasota, ,',
       'west springfield, massachusetts,', 'west linn, ,'], dtype=object)

Подавляющее большинство городов (в частности, по графику распределения стран из практической работы 2) относится к США — это самое часто встречающееся государство. Заполним пропуски модой.

In [19]:
books["country"] = books["country"].fillna(books["country"].mode()[0])

Столбец `Location` нам больше не понадобится, можем его удалить.

In [20]:
books.drop(columns=["Location"], inplace=True)

И, наконец, разделим для удобства числовые и категориальные признаки.

In [21]:
numeric_cols = [
    "ISBN",  # после преобразования LabelEncoder'ом будем считать ISBN числовым признаком
    "Year-Of-Publication",
    "User-ID",
    "Book-Rating",
    "Age",
]

categorial_cols = [
    "Book-Title",
    "Book-Author",
    "Publisher",
    "city",
    "region",
    "country",
]

# Нормировка числовых признаков

Для корректной работы некоторых моделей (например, KNN, нейронные сети) предварительно необходимо провести нормировку некоторых числовых признаков. Поскольку "истинно" числовыми являются признаки `Year-Of-Publication` и `Age`, будем нормировать именно их.

Из результатов второй практики видно, что распределение значений этих признаков похоже на нормальное, поэтому имеет смысл воспользоваться `StandardScaler`.

In [22]:
from sklearn.preprocessing import StandardScaler

for col in ("Year-Of-Publication", "Age"):
    books[col] = StandardScaler().fit_transform(books[col].values.reshape(-1, 1))

На данном этапе мы не сохраняем модели преобразования данных — мы только тестируем разные комбинации моделей, чтобы в дальнейшем составить пайплайн для предсказания рейтинга книги.

# Преобразование категориальных признаков и обучение моделей

Для преобразования категориальных данных в числовые признаки, в том числе названий книг, мы выбрали несколько подходов, такие как Bag of Words, TF-IDF, Word2Vec. Давайте рассмотрим каждый из них.

## Bag of Words

**bag-of-words - подход, при котором текст представляется в матрицы (количества вхождений слов в документы).**

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_title = CountVectorizer(max_features=50)
vectorizer_author = CountVectorizer(max_features=50)
vectorizer_publisher = CountVectorizer(max_features=50)
vectorizer_city = CountVectorizer(max_features=50)
vectorizer_region = CountVectorizer(max_features=50)
vectorizer_country = CountVectorizer(max_features=50)

title_bow = vectorizer_title.fit_transform(books['Book-Title'])
author_bow = vectorizer_author.fit_transform(books['Book-Author'])
publisher_bow = vectorizer_publisher.fit_transform(books['Publisher'])
city_bow = vectorizer_city.fit_transform(books['city'])
region_bow = vectorizer_region.fit_transform(books['region'])
country_bow = vectorizer_country.fit_transform(books['country'])

In [ ]:
title_vocab = vectorizer_title.get_feature_names_out()

print("Title List:")
for token in title_vocab:
    print(token)

Title List:
all
american
amp
an
and
at
book
books
by
classics
club
death
for
from
guide
harlequin
house
how
in
is
life
little
love
man
my
mysteries
mystery
new
night
no
novel
novels
of
on
one
other
paperback
romance
series
star
stories
story
the
time
to
who
with
world
you
your


Вывод представляет собой список токенов, полученных в результате векторизации заголовков книг с использованием CountVectorizer. Каждый токен представляет собой уникальное слово или фразу, найденную в заголовках книг.

In [ ]:
print("Author BOW Matrix:")
author_bow.toarray()

Author BOW Matrix:


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

Объединим преобразованные категориальные признаки с существующими числовыми и обучим модель на линейной регрессии, алгоритме случайного леса и `xgboost`. Качество модели будем оценивать с помощью меры `MSE`.

In [24]:
title_df = pd.DataFrame(title_bow.toarray(), columns=vectorizer_title.get_feature_names_out())
author_df = pd.DataFrame(author_bow.toarray(), columns=vectorizer_author.get_feature_names_out())
publisher_df = pd.DataFrame(publisher_bow.toarray(), columns=vectorizer_publisher.get_feature_names_out())
city_df = pd.DataFrame(city_bow.toarray(), columns=vectorizer_city.get_feature_names_out())
region_df = pd.DataFrame(region_bow.toarray(), columns=vectorizer_region.get_feature_names_out())
country_df = pd.DataFrame(country_bow.toarray(), columns=vectorizer_country.get_feature_names_out())

df = pd.concat([books[numeric_cols], title_df, author_df, publisher_df, city_df, region_df, country_df], axis=1)

del title_bow, author_bow, publisher_bow, city_bow, region_bow, country_bow
del title_df, author_df, publisher_df, city_df, region_df, country_df

gc.collect()  # вызываем сборщик мусора, чтобы освободить память

4

Далее объединим столбцы с одинаковыми именами, которые возникли при объединении столбцов.

In [25]:
duplicate_columns = df.columns[df.columns.duplicated(keep=False)]
combined_columns = {col: df.filter(like=col).sum(axis=1) for col in duplicate_columns}

df.drop(columns=duplicate_columns, inplace=True)

for col_name, col_values in combined_columns.items():
    df[col_name + "_combined"] = col_values

del duplicate_columns, combined_columns
gc.collect()

0

Разделим выборку на обучающую и тестовую.

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Параметры train_test_split
random_state = 42
test_size = 0.2

X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=["Book-Rating"]), df["Book-Rating"], test_size=test_size, random_state=random_state)
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)
y_train.fillna(y_train.mode()[0], inplace=True)
y_test.fillna(y_test.mode()[0], inplace=True)

del df
gc.collect()

0

## BoW + LinearRegression

In [27]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred = list(map(lambda x: round(x), y_pred))

mean_squared_error(y_pred, y_test)

14.178011715189013

MSE для Bag of Words и линейной регрессии — 14.17, RMSE — примерно 3.77. Ошибка предсказания составляет примерно 4 балла.

## BoW + RandomForest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=50, max_depth=15, n_jobs=-1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred = list(map(lambda x: round(x), y_pred))

mean_squared_error(y_pred, y_test)

13.773963768256493

MSE для Bag of Words и линейной регрессии — 13.77, RMSE — примерно 3.71. Ошибка предсказания составляет примерно 4 балла.

## BoW + XGBoost

In [28]:
import xgboost as xgb

model = xgb.XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred = list(map(lambda x: round(x), y_pred))

mean_squared_error(y_pred, y_test)

14.085177570456

MSE для Bag of Words и XGBoost — 14.09, RMSE — примерно 3.75. Ошибка предсказания составляет примерно 4 балла.

## TF-IDF

**tf–idf — это способ векторизации текста, отражающий важность слова в документе, а не только частоту его появления**

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_title = TfidfVectorizer(max_features=50)
vectorizer_author = TfidfVectorizer(max_features=50)
vectorizer_publisher = TfidfVectorizer(max_features=50)
vectorizer_city = TfidfVectorizer(max_features=50)
vectorizer_region = TfidfVectorizer(max_features=50)
vectorizer_country = TfidfVectorizer(max_features=50)

title_tfidf = vectorizer_title.fit_transform(books['Book-Title'])
author_tfidf = vectorizer_author.fit_transform(books['Book-Author'])
publisher_tfidf = vectorizer_publisher.fit_transform(books['Publisher'])
city_tfidf = vectorizer_city.fit_transform(books['city'])
region_tfidf = vectorizer_region.fit_transform(books['region'])
country_tfidf = vectorizer_country.fit_transform(books['country'])

Аналогично преобразуем полученные векторы в датафреймы, объединим числовые и категориальные признаки, избавимся от повторов.

In [25]:
title_df = pd.DataFrame(title_tfidf.toarray(), columns=vectorizer_title.get_feature_names_out())
author_df = pd.DataFrame(author_tfidf.toarray(), columns=vectorizer_author.get_feature_names_out())
publisher_df = pd.DataFrame(publisher_tfidf.toarray(), columns=vectorizer_publisher.get_feature_names_out())
city_df = pd.DataFrame(city_tfidf.toarray(), columns=vectorizer_city.get_feature_names_out())
region_df = pd.DataFrame(region_tfidf.toarray(), columns=vectorizer_region.get_feature_names_out())
country_df = pd.DataFrame(country_tfidf.toarray(), columns=vectorizer_country.get_feature_names_out())

df = pd.concat([books[numeric_cols], title_df, author_df, publisher_df, city_df, region_df, country_df], axis=1)

del title_tfidf, author_tfidf, publisher_tfidf, city_tfidf, region_tfidf, country_tfidf
del title_df, author_df, publisher_df, city_df, region_df, country_df

gc.collect()

4

In [26]:
duplicate_columns = df.columns[df.columns.duplicated(keep=False)]
combined_columns = {col: df.filter(like=col).sum(axis=1) for col in duplicate_columns}

df.drop(columns=duplicate_columns, inplace=True)

for col_name, col_values in combined_columns.items():
    df[col_name + "_combined"] = col_values

del duplicate_columns, combined_columns
gc.collect()

0

Разделим выборку на обучающую и тестовую и приступим к обучению моделей (как и раньше, `LinearRegression`, `RandomForestRegressor` и `XGBoostRegressor`).

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Параметры train_test_split
random_state = 42
test_size = 0.2

X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=["Book-Rating"]), df["Book-Rating"], test_size=test_size, random_state=random_state)
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)
y_train.fillna(y_train.mode()[0], inplace=True)
y_test.fillna(y_test.mode()[0], inplace=True)

del df
gc.collect()

0

## TF-IDF + LinearRegression

In [28]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred = list(map(lambda x: round(x), y_pred))

mean_squared_error(y_pred, y_test)

14.178098997226371

MSE для TF-IDF и линейной регрессии — 14.18, RMSE — примерно 3.77. Ошибка предсказания составляет примерно 4 балла.

## TF-IDF + RandomForest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=50, max_depth=15, n_jobs=-1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred = list(map(lambda x: round(x), y_pred))

mean_squared_error(y_pred, y_test)

13.790683127412379

MSE для TF-IDF и RandomForest — 13.79, RMSE — примерно 3.71. Ошибка предсказания составляет примерно 4 балла.

## TF-IDF + XGBoost

In [29]:
import xgboost as xgb

model = xgb.XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred = list(map(lambda x: round(x), y_pred))

mean_squared_error(y_pred, y_test)

14.081744476986636

MSE для TF-IDF и XGBoost — 14.08, RMSE — примерно 3.75. Ошибка предсказания составляет примерно 4 балла.

## Word2Vec

**W2V — модель, которая фиксирует синтаксическое и семантическое сходство между словами, т.е. слова, близкие по значению, будут помещены рядом**

Обучим модель для каждого столбца и далее получим векторное представление для каждого значения категориального признака, усреднив векторы, полученные для данного значения с помощью Word2Vec (или, в случае отсутствия слов в модели, вернув нулевой вектор).

In [23]:
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
import numpy as np

vector_size = 50

w2v_models = dict()
for col in categorial_cols:
    w2v_models[col] = Word2Vec([str(value).split() for value in books[col].unique()],
                               vector_size=vector_size, window=5, min_count=1, workers=4)

In [24]:
def vectorize_and_average(model, value):
    """
    Возвращает средний вектор для набора слов или нулевой вектор той же
    размерности, если ни одного слов нет в словаре модели
    """
    words = str(value).split()
    vectors = [model.wv[word] for word in words if word in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    return np.zeros(model.vector_size)

df_vectorized = pd.DataFrame()

for col in categorial_cols:
    df_vectorized[col] = books[col].apply(lambda x: vectorize_and_average(w2v_models[col], x))

# далее "распрямим" векторы: пусть каждая координата стоит в отдельном столбце
for col in categorial_cols:
    col_vectors = pd.DataFrame(df_vectorized[col].tolist(), columns=[f"{col}_{i+1}" for i in range(vector_size)])
    df_vectorized = df_vectorized.drop(col, axis=1)
    df_vectorized = pd.concat([df_vectorized, col_vectors], axis=1)
    df_vectorized.dropna(inplace=True)

In [25]:
df = pd.concat([books[numeric_cols], df_vectorized], axis=1)
del df_vectorized, books
gc.collect()

0

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Параметры train_test_split
random_state = 42
test_size = 0.2

df.dropna(inplace=True)

X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=["Book-Rating"]), df["Book-Rating"],
    test_size=test_size, random_state=random_state)

del df
gc.collect()

0

## Word2Vec + LinearRegression

In [29]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred = list(map(lambda x: round(x), y_pred))

mean_squared_error(y_pred, y_test)

14.34815224296029

MSE для Word2Vec и LinearRegression — 14.35, RMSE — примерно 3.79. Ошибка предсказания составляет примерно 4 балла.

## Word2Vec + RandomForest

In [27]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=30, max_depth=10, n_jobs=-1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred = list(map(lambda x: round(x), y_pred))

mean_squared_error(y_pred, y_test)

13.496651682886972

MSE для Word2Vec и RandomForest — 13.5, RMSE — примерно 3.67. Ошибка предсказания составляет примерно 4 балла.

## Word2Vec + XGBoost

In [29]:
import xgboost as xgb

model = xgb.XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred = list(map(lambda x: round(x), y_pred))

mean_squared_error(y_pred, y_test)

13.735558109425234

MSE для Word2Vec и XGBoost — 13.73, RMSE — примерно 3.7. Ошибка предсказания составляет примерно 4 балла.

# Вывод

Наиболее удачной при текущей предобработке является использование векторизатора Word2Vec и модели RandomForestRegressor — эта комбинация даёт наименьшую среднеквадратическую ошибку.

Результат можно улучшить, например, подобрав более удачные гиперпараметры RandomForest, увеличив размер вектора Word2Vec. Однако, поскольку все 9 комбинаций дают примерно одинаковый результат, больший вклад даёт более качественная предобработка

**Возможный пайплайн:**

1.   Удаление признаков `Image-URL-*`
2.   Разделение `Location` на `city`, `region`, `country`, удаление `Location`
3.   Заполнение отсутствующих значений: мода для `Year-Of-Publication`, `city`, `region`, `country`, медиана для `Age` (на основе тренировочного датасета)
4.   Применение обученного `LabelEncoder` для `ISBN`
5.   Применение обученных `StandardScaler` для `Year-Of-Publication` и `Age`
6.   Применение обученных моделей `Word2Vec` для векторизации
7.   "Распрямление" векторов `Word2Vec`
8.   Предсказание с помощью обученного `RandomForestRegressor`